# **Dealing with Async. Activity in the Jupyter Notebook**

## About this presentation

* Feedback on Jupyter as control platform for both machine and beamlines.
* The following context is extracted from a POC (subproject of the SOLEIL "BLISS" project).
* Focus on async. activity in the notebook.   

## Initial motivation

* Mixing CLI & GUI for machine and/or beamline controls?
* The idea was to mix both CLI and GUI into a single environment in order to benefit from the best of both worlds.
    * A pure CLI is convenient and powerful. But, provides no or very poor overview of the underlying systems state. 
    * With a GUI, no need to learn commands and options. However, GUIs are painful to maintain not so often ergonomic. 
* CLI & GUI complement each other. It consequently seems interesting to have both in a unique context.  

## Why Jupyter?
* Jupyter naturally solves a big part of the problem and its web nature provides remote access for free.

## Main problems to be solved
* Async. activity: how to refresh GUIs and plots in the background?
* Cell specific outputs: how route outputs to a specific cell?
* Large data handling: can we display large detector images?

### Let's start with async. activity and cell specific outputs

Jupyter async. features based on underlying the Tornado.IOLoop - who's a wrapper around the Python asyncio event loop.

### `AsyncCallback`: a base class for async. callbacks relying on the tornado.ioloop

In [ ]:
import tornado.ioloop

In [ ]:
class AsyncCallback(object): 
    def __init__(self):
        self.__cbc = 0
        self.__pcb = tornado.ioloop.PeriodicCallback(self.__cbf, 1000.)

    def start(self):
        self.__pcb.start()
        
    def stop(self):
        self.__pcb.stop()
        
    def is_running(self):
        return self.__pcb.is_running()
        
    def set_callback_period(self, p):
        p = max(0.1, p) 
        p = min(p, 1.0)
        was_running =  self.__pcb.is_running()
        self.__pcb.stop()
        self.__pcb = tornado.ioloop.PeriodicCallback(self.__cbf, p * 1000.)
        if was_running:
            self.__pcb.start()

    def __cbf(self):
        self.__cbc += 1
        self.cbf(self.__cbc)
        
    def cbf(self, cbc):
        raise Exception("AsyncCallback: default callback impl. called!")

### `AsyncOutputV1`: an async. callback executing a simple print

In [ ]:
class AsyncOutputV1(AsyncCallback): 
    def __init__(self):
        AsyncCallback.__init__(self)

    def cbf(self, cbc):
        print('AsyncOutputV1.cb-call #{:02d}'.format(cbc))

In [ ]:
ao1 = AsyncOutputV1()

In [ ]:
ao1.start()

In [ ]:
# another cell
print("this is the 'current' cell!")

In [ ]:
# another cell
print("this is the 'current' cell!")

In [ ]:
ao1.stop()

### `AsyncOutputV2`: an async. callback routing its outputs to a "specific" cell output

#### `CellOutput`
A Python context providing a way to route outputs to a "particular" cell: the one that was the "current cell" when the CellOutput instance has been created.

This is a ipython low level implementation based on a simple idea: snapshot the initial context at instanciation, then, when whe want to output something, simply save the current context, activate the initial one, output some content, then restore previous one. Here we use a Python context (and the 'with' keyword) implement the 'set/restore context' mechanism properly. 

In [ ]:
import sys
from IPython.display import clear_output

In [ ]:
class CellOutput(object):
    def __init__(self):
        k = get_ipython().kernel
        self._ident = k._parent_ident
        self._header = k.get_parent()
        self._save_context = None

    def __enter__(self):
        kernel = get_ipython().kernel
        self._save_context = (kernel._parent_ident, kernel.get_parent())
        sys.stdout.flush()
        sys.stderr.flush()
        kernel.set_parent(self._ident, self._header)

    def __exit__(self, etype, evalue, tb):
        sys.stdout.flush()
        sys.stderr.flush()
        kernel = get_ipython().kernel
        kernel.set_parent(*self._save_context)
        return False

    def clear_output(self):
        with self:
            clear_output()

#### `AsyncOutputV2` impl.

In [ ]:
class AsyncOutputV2(AsyncCallback): 
    def __init__(self):
        AsyncCallback.__init__(self)
        self.__outpout = CellOutput()
        
    def cbf(self, cbc):
        self.__outpout.clear_output()
        with self.__outpout:
            print('AsyncOutputV2.cb-call #{:02d}'.format(cbc))
                
    def clear_output(self):
        self.__outpout.clear_output()

In [ ]:
ao2 = AsyncOutputV2()

In [ ]:
ao2.start()

In [ ]:
# another cell
print("this is the 'current' cell!")

In [ ]:
# another cell
print("this is the 'current' cell!")

In [ ]:
ao2.stop()

In [ ]:
ao2.clear_output()

### `AsyncOutputV3`: an async. callback using (ipy)widgets

In [ ]:
import ipywidgets as ipw

In [ ]:
class AsyncOutputV3(AsyncCallback): 
    def __init__(self):
        AsyncCallback.__init__(self)
        self.__int_text = ipw.IntText(value=0, description='Call #', disabled=True)
        self.__progress = ipw.IntProgress(value=0, min=0, max=99, description='Progress')
        b = ipw.Button(tooltip='Start Async. Activity', icon='play', layout=ipw.Layout(height='auto', width='auto'))
        b.on_click(self.__ctrl_button_clicked)
        self.__cbp_slider = ipw.FloatSlider(value = 1.0, min=0.1, max=1.0, description='Period (in s):')
        self.__cbp_slider.observe(self.__period_changed, names='value')
        self.__hbox = ipw.HBox([b, self.__int_text, self.__progress, self.__cbp_slider])
        display(self.__hbox)
        
    def __ctrl_button_clicked(self, b):
        if not self.is_running():
            self.start()
            b.icon = 'stop'
        else:
            self.stop()
            b.icon = 'play'
            
    def __period_changed(self, change):
        self.set_callback_period(change['new'])
        
    def cbf(self, cbc):
        self.__int_text.value = cbc % 100
        self.__progress.value = cbc % 100

In [ ]:
print("foo")

In [ ]:
ao3 = AsyncOutputV3()

In [ ]:
ao4 = AsyncOutputV3()

## `IcePapController`: a motor controller for the Jupyter notebook

In [ ]:
import time
import tango

class IcePapController(AsyncCallback):
    
    class IcePapWidgets(object):
        def __init__(self, device, target_position):
            self.actuator = tango.DeviceProxy(device)
            current_position = self.actuator.Position
            self.forward = target_position > current_position
            children_layout = list()
            self.unit = self.actuator.get_attribute_config('Position').unit
            try:
                self.alias = self.actuator.alias()
            except:
                self.alias = device
            lbt = "moving {} from {:.2f} to {:.2f} {} @ {:.2f} {}/s: ".format(self.alias,
                                                                              current_position,
                                                                              target_position,
                                                                              self.unit,
                                                                              self.actuator.Velocity,
                                                                              self.unit)
            lb = ipw.Label(value=lbt)
            self.slb = ipw.Label(value=" [{}]".format(self.actuator.state()), layout=self.l11a())
            children_layout.append(ipw.HBox([lb, self.slb]))
            widgets = list()
            self.bls = ipw.Checkbox(value=self.actuator.HardLimitHigh,
                                    description='bls',
                                    indent=False,
                                    layout=self.l01a())
            self.bls.observe(self.on_cb_clicked, names='value')
            widgets.append(self.bls)
            will_move = abs(current_position - target_position) > 0.
            if will_move:
              self.pgb = ipw.FloatProgress(value=current_position if self.forward else target_position,
                                           min=min(current_position, target_position),
                                           max=max(current_position, target_position))
            else:
              self.pgb = ipw.FloatProgress(value=100,
                                           min=0,
                                           max=100)    
            self.pgb.style.bar_color = self.device_state_to_widget_color(self.actuator.state())                      
            widgets.append(self.pgb)
            self.fls = ipw.Checkbox(value=self.actuator.HardLimitLow,
                                    description='fls',
                                    indent=False,
                                    layout=self.l01a())
            self.fls.observe(self.on_cb_clicked, names='value')
            widgets.append(self.fls)
            lb = ipw.Label(value='', layout=self.l01a(width='30px'))
            widgets.append(lb)
            lbt = " {:.2f} {}".format(current_position, self.unit)
            self.plb = ipw.Label(value=lbt, layout=self.l01a())
            widgets.append(self.plb)
            lb = ipw.Label(value='', layout=self.l01a(width='30px'))
            widgets.append(lb)
            b = ipw.Button(tooltip='Abort Motion', icon='stop', layout=ipw.Layout(height='auto', width='auto'))
            b.on_click(self.abort)
            widgets.append(b)
            self.status_widgets = ipw.HBox(widgets)
            children_layout.append(self.status_widgets)
            self.layout = ipw.VBox(children_layout, layout=self.l01a())

        def on_cb_clicked(self, change):
            self.update_limit_switches()

        def get_pos_and_state(self):
            try:
                current_position = self.actuator.Position
                current_state = self.actuator.state()
            except:
                current_position = float('nan')
                current_state = Tango.DevState.UNKNOWN
            return current_position, current_state

        def update_progress_bar(self, current_position, current_state, motion_done=False):
            if motion_done:
                bad_states = [tango.DevState.UNKNOWN, tango.DevState.FAULT]
                self.pgb.value = current_position if current_state in bad_states else self.pgb.max
            else:
                self.pgb.value = current_position
            self.pgb.style.bar_color = self.device_state_to_widget_color(current_state)

        def update_limit_switches(self):
            self.bls.value = self.actuator.HardLimitLow
            self.fls.value = self.actuator.HardLimitHigh

        def update_state(self, current_state):
            self.slb.value = "[{}]".format(str(current_state))

        def update(self, aborted=False):
            motion_done = self.actuator.state() != tango.DevState.MOVING
            current_position, current_state = self.get_pos_and_state()
            self.update_progress_bar(current_position, current_state, motion_done)
            self.update_limit_switches()
            self.update_state(current_state)
            if motion_done:
                plb_value = "\t{}[{} is @ {:.2f} {}]".format('*** ABORTED ***\t' if aborted else '',
                                                             self.alias,
                                                             current_position,
                                                             self.unit)
            else:
                plb_value = "\t{:.2f} {}".format(current_position, self.unit)
            self.plb.value = plb_value

        def abort(self, b=None):
            self.actuator.Abort()
            self.update(aborted=True)
            
        def l01a(cls, width='auto', *args, **kwargs):
            return ipw.Layout(flex='0 1 auto', width=width, *args, **kwargs)
            
        def l11a(cls, width='auto', *args, **kwargs):
            return ipw.Layout(flex='1 1 auto', width=width, *args, **kwargs)
    
        def device_state_to_widget_style(cls, s):
            if s in [tango.DevState.STANDBY, tango.DevState.ON]:
                return 'warning'
            if s in [tango.DevState.RUNNING]:
                return 'success'
            if s in [tango.DevState.MOVING]:
                return 'primary'
            if s in [tango.DevState.FAULT, tango.DevState.UNKNOWN]:
                return 'danger'
            if s in [tango.DevState.ALARM]:
                return 'danger'
            return ''

        def device_state_to_widget_color(cls, s):
            if s in [tango.DevState.INIT]:
                return '#CCCC7A'
            if s in [tango.DevState.STANDBY]:
                return '#FFFF00'
            if s in [tango.DevState.ON, tango.DevState.OPEN, tango.DevState.EXTRACT]:
                return '#00FF00'
            if s in [tango.DevState.OFF, tango.DevState.CLOSE, tango.DevState.INSERT]:
                return '#FFFFFF'
            if s in [tango.DevState.MOVING]:
                return '#80A0FF'
            if s in [tango.DevState.RUNNING]:
                return '#228B22'
            if s in [tango.DevState.FAULT]:
                return '#FF0000'
            if s in [tango.DevState.UNKNOWN]:
                return '#808080'
            if s in [tango.DevState.ALARM]:
                return '#FF8C00'
            if s in [tango.DevState.DISABLE]:
                return '#FF00FF'
            return '#808080'

    def __init__(self):
        AsyncCallback.__init__(self)
        self.layout = None
        self.actuator_widgets = None

    def __update_widgets(self, aborted=False):
        for w in self.actuator_widgets.values():
            w.update(aborted)
            
    def goto(self, move_requests):
        """requests: {actuator1:position1, actuator2:position2, ...}"""
        # setup widgets for each actuator
        self.actuator_widgets = {k: IcePapController.IcePapWidgets(k, v) for k, v in move_requests.items()}
        b = ipw.Button(tooltip='Abort motions', icon='stop', layout=ipw.Layout(height='auto', width='auto'))
        b.on_click(self.abort)
        widgets = [v.layout for _, v in self.actuator_widgets.items()]
        #widgets.append(b)
        self.layout = ipw.VBox(widgets)
        display(self.layout)
        self.start()        
        for k, v in move_requests.items():
            tango.DeviceProxy(k).Position = v

    def abort(self, b=None):
        self.stop()
        for w in self.actuator_widgets.values():
            w.abort()
            
    def cbf(self, cbc):
        self.__update_widgets()

In [ ]:
ipc = IcePapController()

In [ ]:
ipc.goto({"m1":100, "m2":200})

## A `SPEC` like CLI for the `IcePapController` based on the `IPython magics`

Define 'amove' magic with the following syntax: **amove motor-name target-position** 

In [ ]:
from IPython.core.magic import Magics, magics_class, line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring

@magics_class
class MyMagics(Magics):

    @magic_arguments()
    @argument('movable', help='fully specified movable name or alias')
    @argument('position', help='target position')
    @line_magic
    def mv(self, line):
        ns = parse_argstring(self.mv, line)
        ipc = IcePapController()
        ipc.goto({ns.movable:float(ns.position)})
        
get_ipython().register_magics(MyMagics)

In [ ]:
mv m1 0

## jupyter for DAQ configuration, control and monitoring
Dynamic interface based on introspection of the scan configuration. 

* **scan parameters**
<img src="./tango_meeting_florence_0617/resources/images/fs_gui_params.png">
* **sensors selection**
<img src="./tango_meeting_florence_0617/resources/images/fs_gui_sensors.png">
* **scan actors monitoring**
<img src="./tango_meeting_florence_0617/resources/images/fs_gui_devices_monitoring.png">


# **The `jupyTango` environment**

jupyTango defines two jupyter magics (more to come): `%tango_monitor` and `%plot_tango_attribute`.

The`%plot_tango_attribute` (or `%pta`) generates a static/synchronous plot of any tango attribute (snapshot).

In [ ]:
# ugly but mandatory: select the context in which we are running: NOTEBOOK or LAB
import os
os.environ["JUPYTER_CONTEXT"] = "LAB"

The `%tango_monitor` - or its alias `%tgm` - provides us with a live/asynchronous monitor for any tango attribute.

In [ ]:
tgm sys/tg_test/nl1/double_image_ro

In [ ]:
tgm -w 1200 sys/tg_test/nl1/double_scalar

## Mix. a Tango monitor with an IcePapController

In [ ]:
tgm -w 1200 -p 0.25 m1/Position

In [ ]:
mv m1 200

In [ ]:
tango.DeviceProxy("m1").Velocity = 10.

In [ ]:
!which python